In [2]:
import piplite
await piplite.install(["ipywidgets", "bqplot", "plotly"])

In [3]:
import ipywidgets

In [4]:
#%pylab inline
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

In [14]:
T = 1000
dt = 0.1
figs = []

@widgets.interact(log_λ=(-4.0,1.0), D=(5,120), log_σ=(-3.0, 1.0), log_k=(-3.0,2.0))
def run(log_λ=0.0, D=100, log_σ=0.0, log_k=0.0, stochastic=True):
    λ = np.exp(log_λ)
    k = np.exp(log_k)
    if stochastic:
        z = np.sqrt(dt)*np.random.randn(2,T)*np.exp(log_σ)
    x, y = 1.0, 0.0
    xs, ys = [], []
    for i in range(T):
        dx = -λ*x - y
        dy = -λ*y + x
        if i > D:
            dx -= k*ys[i-D]
        x = x + dt*dx + (z[0,i] if stochastic else 0.0)
        y = y + dt*dy + (z[1,i] if stochastic else 0.0)
        xs.append(x)
        ys.append(y)
    xy = np.array([xs,ys]).T

    L = np.zeros((2+D, 2+D), 'f') # x, y, y history
    L[0,0] = 1.0 + dt*(-λ)  # x -> x
    L[0,1] = dt*(-1)  # y -> x
    L[0,-1] = dt*(-k) # y(t-D) -> x
    L[1,0] = dt # x -> y
    L[1,1] = 1.0 + dt*(-λ) # y -> y
    for i in range(D):
        #L[2+i,2+i] = -1.0
        L[2+i,1+i] = 1.0
    
    
    fig, ax = plt.subplots(2,2, figsize=(15,9));
    ax[0,0].plot(xy); ax[0,0].grid(1); #ax[0,0].set_ylim([-5,5]); ax[0,0].axvline(D, color='g')
    ax[0,1].imshow(L)
    e,v = np.linalg.eig(L); print(np.abs(e.real)[:4])
    ax[1,0].plot(e.real, 'k');    ax[1,0].plot(e.imag, 'r'); ax[1,0].grid(1)
    ax[1,1].plot(np.abs(e)); ax[1,1].grid(1); ax[1,1].axhline(1.0, color='r')
    plt.show()
    figs.append(fig)
    if len(figs)>2:
        plt.close(figs[0])
        del figs[0]

interactive(children=(FloatSlider(value=0.0, description='log_λ', max=1.0, min=-4.0), IntSlider(value=100, des…